In [7]:
%reset -f
import h5py,os,time
import numpy as np
import numpy.matlib
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.linalg import block_diag

#set physical constants
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kA=6.022e23 #Avogadro constant [1/mol]
kB=1.381e-23 #Boltzmann's constant [J/K]

def export_mesh(infile):    
    print('Reading group mesh from %s'%infile)
    with h5py.File(infile,'r') as file:
        nodes=np.array(file['mesh/nodes'])
        elements=np.array(file['mesh/elements'])
        faces=np.array(file['mesh/faces'])
                
        mask_e=np.array(file['mesh/mask_elements'])
        mask_n=np.array(file['mesh/mask_nodes_first_kind_bc'])
        mask_f=np.array(file['mesh/mask_faces_third_kind_bc'])
        
        mask_electrolyte=np.array(file['mesh/mask_electrolyte'])
        mask_solid=np.array(file['mesh/mask_solid'])
        mask_stern=np.array(file['mesh/mask_stern'])
        dist_scale=np.array(file['mesh/dist_scale'])
            
        #compute middle point of each element (efficiency to be improved)
        nelem=len(elements[:,0])
        midpoints=np.zeros((nelem,3))
        for i in range(0,nelem):
            for j in range(0,3):
                midpoints[i,j]=.25*sum(nodes[elements[i,:],j])
        
        nind_e=np.unique(elements[mask_e].flatten(order='C'))
        print('THE NUMBER OF COMPUTATIONAL NODES IS:',len(nind_e))
        print('THE NUMBER OF COMPUTATIONAL ELEMENTS IS:',np.count_nonzero(mask_e))
        print('THE NUMBER OF NODES ON S1 IS:',np.count_nonzero(mask_n))
        print('THE NUMBER OF ELEMENTS ON S2 IS:',np.count_nonzero(mask_f))
        print('DISTANCE SCALING FACTOR IS:',dist_scale)
        print('')

        #remember to apply distance scaling coefficient 
        nodes=nodes*dist_scale
        midpoints=midpoints*dist_scale
        mesh={'elements':elements,'nodes':nodes,'faces':faces,
              'mask_elements':mask_e,'mask_nodes_first_kind_bc':mask_n,'mask_faces_third_kind_bc':mask_f,
              'mask_electrolyte':mask_electrolyte,'mask_solid':mask_solid,'mask_stern':mask_stern,
              'mid_points':midpoints}
    
    return mesh

def export_phys(infile):
    print('Reading group physics from %s'%infile)
    with h5py.File(infile,'r') as file:
        rel_perm_electrolyte=np.array(file['physics/rel_perm_electrolyte'])
        rel_perm_solid=np.array(file['physics/rel_perm_solid'])
        diffusion_electrolyte=np.array(file['physics/diffusion_electrolyte'])
        diffusion_stern=np.array(file['physics/diffusion_stern'])
        mobility_electrolyte=np.array(file['physics/mobility_electrolyte'])
        mobility_stern=np.array(file['physics/mobility_stern'])
        temp_K=np.array(file['physics/temperature'])
        cinf=np.array(file['physics/ion_conc_inf'])
        zval=np.array(file['physics/ion_val'])
        sigma_ss=np.array(file['physics/sigma_ss'])
        
        print('RELATIVE PERMITTIVITY OF ELECTROLYTE IS:',rel_perm_electrolyte)
        print('RELATIVE PERMITTIVITY OF SOLID IS:',rel_perm_solid)
        print('DIFFUSION COEFFICIENT OF ELECTROLYTE IS:',diffusion_electrolyte)
        print('DIFFUSION COEFFICIENT OF STERN LAYER IS:',diffusion_stern)
        print('ION MOBILITY IN ELECTROLYTE IS:',mobility_electrolyte)
        print('ION MOBILITY IN STERN LAYER IS:',mobility_stern)
        print('TEMPERATURE IS:',temp_K)
        print('ION COCENTRATION AT INFINITY IS:',cinf)
        print('ION VALENCE IS:',zval)
        print('SURFACE CHARGE DENSITY FOR STEADY STATE IS:',sigma_ss)
        print('')
        
        phys={'rel_perm_electrolyte':rel_perm_electrolyte,'rel_perm_solid':rel_perm_solid,
              'diffusion_electrolyte':diffusion_electrolyte,'diffusion_stern':diffusion_stern,
              'mobility_electrolyte':mobility_electrolyte,'mobility_stern':mobility_stern,
              'temperature':temp_K,'ion_conc_inf':cinf,'ion_val':zval,
              'sigma_ss':sigma_ss}
    
    return phys

# def build_K()

def set_PB(mesh,phys,unod=0,dist_scale=1):
    global cx,cy,cz
    global alpha_x,alpha_y,alpha_z
    global beta_x,beta_y,beta_z
    global gamma_x,gamma_y,gamma_z
    global an,fn,gs,qs,sn
    
    nelem=len(mesh['elements'])
    nnode=len(mesh['nodes'])
    nface=len(mesh['faces'])
    
    mask_electrolyte=mesh['mask_electrolyte']
    mask_solid=mesh['mask_solid']
    mask_stern=mesh['mask_stern']
    
    rel_perm_electrolyte=phys['rel_perm_electrolyte']
    rel_perm_solid=phys['rel_perm_solid']
    temp_K=phys['temperature']
    cinf=phys['ion_conc_inf']
    zval=phys['ion_val']
    sigma_ss=phys['sigma_ss']
        
    c=np.zeros(nelem)
    alpha_x=np.zeros(nelem)
    alpha_y=np.zeros(nelem)
    alpha_z=np.zeros(nelem)

    gamma_x=np.zeros(nelem)
    gamma_y=np.zeros(nelem)
    gamma_z=np.zeros(nelem)

    beta_x=np.zeros(nelem)
    beta_y=np.zeros(nelem)
    beta_z=np.zeros(nelem)

    an=np.zeros(nnode)
    fn=np.zeros(nnode)
    gs=np.zeros(nface)
    qs=np.zeros(nface)
    sn=np.zeros(nnode)

    c[mask_electrolyte]=perm0*rel_perm_electrolyte
    c[mask_solid]=perm0*rel_perm_solid
    gs[mask_stern]=sigma_ss/dist_scale
    
    ze=zval*echarge #ion valence times elementary charge
    unod_scale=ze*unod/kB/temp_K #scaled potential on nodes
    an[:]=2*ze**2*kA*cinf/kB/temp_K*np.cosh(unod_scale)/dist_scale**2
    fn[:]=-2*ze*kA*cinf*(np.sinh(unod_scale)-np.cosh(unod_scale)*unod_scale)/dist_scale**2
    
    return

def compute_u(nodes,elements,faces,unod,flag=3):
    print('Computing field and field gradient')
    start=time.time()
    
    if flag==3:
        nelem=len(elements)
        u=np.zeros((nelem,4))
        Je=np.ones((4,4))
        for i in range(nelem): #consider parallelization
            nind=elements[i,:]        
            Je[1:,:]=nodes[nind,:].T
            invJe=np.linalg.inv(Je)

            xr=.25*sum(nodes[nind,0])
            yr=.25*sum(nodes[nind,1])
            zr=.25*sum(nodes[nind,2])
            u[i,0]=sum((invJe[:,0]+invJe[:,1]*xr+invJe[:,2]*yr+invJe[:,3]*zr)*unod[nind])
            u[i,1]=sum(unod[nind]*invJe[:,1])
            u[i,2]=sum(unod[nind]*invJe[:,2])
            u[i,3]=sum(unod[nind]*invJe[:,3])
    else:
        nface=len(faces)
        u=np.zeros((nface,4))
        Je=np.ones((3,3))
        for i in range(1): #consider parallelization
            nind=faces[i,:]
            R=rotate_triangle(nodes[nind,:])
            nodes_new=R.dot(nodes[nind,:].T).T
            
            Je[1:,:]=nodes_new[:,:-1].T
            invJe=np.linalg.inv(Je)
            
            xr=sum(nodes_new[:,0])/3
            yr=sum(nodes_new[:,1])/3
            u[i,0]=sum((invJe[:,0]+invJe[:,1]*xr+invJe[:,2]*yr)*unod[nind])
            
            dudx=sum(unod[nind]*invJe[:,1])
            dudy=sum(unod[nind]*invJe[:,2])
            u[i,1:]=np.linalg.inv(R).dot([dudx,dudy,0])
#             print(u[i,:])
            
# #             print('R')
# #             print(R)
# #             print('nodes_new')
# #             print(nodes_new)
        
# #         A=np.zeros((3,3))
# #         R=np.zeros((3,3))
# #         nodes_new=np.zeros((3,3))
# #         Je=np.ones((3,3))
# #         for i in range(1): #consider parallelization
# #             nind=faces[i,:]
# #             xnod=nodes[nind,0]
# #             ynod=nodes[nind,1]
# #             znod=nodes[nind,2]
            
# #             a=[xnod[1]-xnod[0],ynod[1]-ynod[0],znod[1]-znod[0]]
# #             b=[xnod[2]-xnod[1],ynod[2]-ynod[1],znod[2]-znod[1]]
# #             n=np.cross(a,b)
# #             area=np.linalg.norm(n)/2
            
# #             n=n/np.linalg.norm(n)
# #             z=np.array([0,0,1])
# #             k=np.cross(n,z)
# #             k=k/np.linalg.norm(k)
            
# #             cosb=np.dot(n,z)
# #             sinb=np.sqrt(1-cosb**2)
            
# #             A[0,:]=[0,-k[2],k[1]]
# #             A[1,:]=[k[2],0,-k[0]]
# #             A[2,:]=[-k[1],k[0],0]
# #             R=np.eye(3)+A.dot(sinb)+A.dot(A.dot(1-cosb))
            
# #             nodes_new[0,:]=R.dot([xnod[0],ynod[0],znod[0]])
# #             nodes_new[1,:]=R.dot([xnod[1],ynod[1],znod[1]])
# #             nodes_new[2,:]=R.dot([xnod[2],ynod[2],znod[2]])
            
# #             print('R')
# #             print(R)
#             print('nodes_new')
#             print(nodes_new)
#             xnod=nodes[nind,0]
#             ynod=nodes[nind,1]
#             znod=nodes[nind,2]
#             a=[xnod[1]-xnod[0],ynod[1]-ynod[0],znod[1]-znod[0]]
#             b=[xnod[2]-xnod[1],ynod[2]-ynod[1],znod[2]-znod[1]]
#             c=[xnod[0]-xnod[2],ynod[0]-ynod[2],znod[0]-znod[2]]
#             print('0: len(a) %.2e len(b) %.2e len(c) %.2e'%(np.linalg.norm(a),np.linalg.norm(b),np.linalg.norm(c)))
#             xnod=nodes_new[:,0]
#             ynod=nodes_new[:,1]
#             znod=nodes_new[:,2]
#             a=[xnod[1]-xnod[0],ynod[1]-ynod[0],znod[1]-znod[0]]
#             b=[xnod[2]-xnod[1],ynod[2]-ynod[1],znod[2]-znod[1]]
#             c=[xnod[0]-xnod[2],ynod[0]-ynod[2],znod[0]-znod[2]]
#             print('1: len(a) %.2e len(b) %.2e len(c) %.2e'%(np.linalg.norm(a),np.linalg.norm(b),np.linalg.norm(c)))
#             print('')

            

    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')
    return u

def rotate_triangle(nodes):
    A=np.zeros((3,3))
    R=np.zeros((3,3))
    
    xnod=nodes[:,0]
    ynod=nodes[:,1]
    znod=nodes[:,2]
    
    a=[xnod[1]-xnod[0],ynod[1]-ynod[0],znod[1]-znod[0]]
    b=[xnod[2]-xnod[1],ynod[2]-ynod[1],znod[2]-znod[1]]
    n=np.cross(a,b)
    area=np.linalg.norm(n)/2
    
    n=n/np.linalg.norm(n)
    z=np.array([0,0,1])
    k=np.cross(n,z)
    k=k/np.linalg.norm(k)
    
    cosb=np.dot(n,z)
    sinb=np.sqrt(1-cosb**2)
    
    A[0,:]=[0,-k[2],k[1]]
    A[1,:]=[k[2],0,-k[0]]
    A[2,:]=[-k[1],k[0],0]
    R=np.eye(3)+A.dot(sinb)+A.dot(A.dot(1-cosb))
    
    return R
    
def solve_unod(K,b):
    print('Calling sparse linear system solver')
    start=time.time()
    K.eliminate_zeros()
    unod=spsolve(K,b)
    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')
    
    return unod

if __name__=='__main__':
    mesh=export_mesh('sphere_s61.hdf5')
    phys=export_phys('sphere_s61.hdf5')
    set_PB(mesh,phys,unod=0,dist_scale=1)
    
    nodes=mesh['nodes']
    elements=mesh['elements']
    faces=mesh['faces']
    unod=np.zeros(len(elements))
    u=compute_u(nodes,elements,faces,unod,flag=2)
    print('Done')


Reading group mesh from sphere_s61.hdf5
THE NUMBER OF COMPUTATIONAL NODES IS: 24157
THE NUMBER OF COMPUTATIONAL ELEMENTS IS: 135851
THE NUMBER OF NODES ON S1 IS: 5334
THE NUMBER OF ELEMENTS ON S2 IS: 2044
DISTANCE SCALING FACTOR IS: 1000000000.0

Reading group physics from sphere_s61.hdf5
RELATIVE PERMITTIVITY OF ELECTROLYTE IS: 78.5
RELATIVE PERMITTIVITY OF SOLID IS: 4.5
DIFFUSION COEFFICIENT OF ELECTROLYTE IS: 2e-09
DIFFUSION COEFFICIENT OF STERN LAYER IS: 1.9999999999999998e-10
ION MOBILITY IN ELECTROLYTE IS: 5e-08
ION MOBILITY IN STERN LAYER IS: 5e-09
TEMPERATURE IS: 298.0
ION COCENTRATION AT INFINITY IS: 100.0
ION VALENCE IS: 1.0
SURFACE CHARGE DENSITY FOR STEADY STATE IS: 0.01

Computing field and field gradient
[0. 0. 0. 0.]
nodes_new
[[ 0.39316019 -0.20294498  3.910953  ]
 [ 0.07508542  0.4315981   3.910953  ]
 [-0.10660463 -0.17971066  3.91095301]]
0: len(a) 7.10e-01 len(b) 6.38e-01 len(c) 5.00e-01
1: len(a) 7.10e-01 len(b) 6.38e-01 len(c) 5.00e-01

Time elapsed  0.00159239768

In [9]:
#Remarks
#check coefficients against equations
#be careful of copying and reassign values in numpy array

#export frequencies
